<a href="https://colab.research.google.com/github/mahananda123/SignLanguageRecognitionUsingDeepLEarning/blob/main/SignLanguageRecognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.15.0


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16

In [ ]:
import tensorflow as tf

# Specify the path to your custom dataset
data_path = '/content/drive/MyDrive/CNN Architecture/original_images'

# Load the data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    labels='inferred',
    label_mode='categorical',
    image_size=(225,225),
    batch_size=50,
    validation_split=0.2,  # Use validation_split for splitting into training and validation sets
    subset='training',  # Subset for training data
    seed=42
)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    labels='inferred',
    label_mode='categorical',
    image_size=(225,225),
    batch_size=20,
    validation_split=0.2,
    subset='validation',  # Subset for validation data
    seed=42
)


Found 35142 files belonging to 36 classes.
Using 28114 files for training.
Found 35142 files belonging to 36 classes.
Using 7028 files for validation.


In [ ]:
# Preprocess the data
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))

In [ ]:

# Load the pre-trained VGG16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(225, 225, 3))

# Freeze the pre-trained layers so they are not updated during training
base_model.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Create the model
model = Sequential()
model.add(Conv2D(16, kernel_size=(2,2), activation='relu', input_shape=(225, 225, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(225, 225, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(36, activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model_hist=model.fit(train_data,validation_data=(test_data),epochs=70)

Epoch 1/70
 10/563 [..............................] - ETA: 38:25 - loss: 3.5940 - accuracy: 0.0300

In [ ]:
# Evaluate the model
model.evaluate(test_data)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(model_hist.history['loss'])
plt.plot(model_hist.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'],loc='upper left')
plt.show()

In [ ]:
plt.plot(model_hist.history['accuracy'])
plt.plot(model_hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np

# Assuming you have a test dataset named test_data
test_images = []  # List to store test images

# Assuming test_data is a tf.data.Dataset
for images, labels in test_data:
    test_images.append(images)

# Concatenate test images into a single tensor
test_images = tf.concat(test_images, axis=0)

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Now you have the predicted labels for your test data
print("Predicted Labels:", predicted_labels)


In [ ]:
import tensorflow as tf
from sklearn.metrics import f1_score

# Convert test_data to a NumPy array or TensorFlow tensor
test_images = []
test_labels = []
for images, labels in test_data:
    test_images.append(images)
    test_labels.append(labels)

test_images = tf.concat(test_images, axis=0)
true_labels = tf.argmax(tf.concat(test_labels, axis=0), axis=1)

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = tf.argmax(predictions, axis=1)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')
print("F1 Score:", f1)


In [ ]:
print(len(predicted_labels))

In [ ]:
model_hist.save('research')

In [ ]:
from colabcode import Colabcode
Colabcode(port=10000)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
5.354825623000011
GPU (s):
0.14207483399999887
GPU speedup over CPU: 37x
